# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [5]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

## Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [ ]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCylceConfig(
    budget=0.01, # the budget for the forge cycle in dollas
    n_agents_in_population=2, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
)

best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:21<00:00, 10.79s/it]
Agent initialization cost: 0.006005505
Population initialized with 2 agents
Initialization of 2 agents took 0:00:21.602633
Budget left after initialization: 0.003994495 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 3231.36it/s]
Agent agent-02268d03-c42e-48ad-9dcb-de8afff5ba9d fitness: 0.7943794815224912, cost: 0.0
Agent agent-3dc2b205-3012-4422-9701-70e9c8f785b2 fitness: 0.6989944337295713, cost: 0.0
Evaluation took 0:00:00.003304 for a total cost of 0.0 $


# Agent ID: agent-02268d03-c42e-48ad-9dcb-de8afff5ba9d
## Fitness: 0.7943794815224912
```mermaid 
graph LR
	Start_Node[start_node] --> Llm1(LLM1)
	Llm1(LLM1) --> Llm2(LLM2)
	Llm2(LLM2) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a network of specialized LLM nodes. Your task is to contribute meaningfully to the problem-solving process while communicating effectively with other LLM nodes. The process begins with the interpretation of the problem, followed by the solution generation. Each node has a distinct role: LLM1: Interpret and extract key information from the math word problem. LLM2: Perform calculations based on the extracted information and generate a solution.
##### LLM1
You are tasked with interpreting a math word problem. Read the problem carefully and identify key elements such as numbers, operations, and relationships. Provide a concise summary that outlines these key details, ensuring to clarify any ambiguities or complexities in the problem. This summary will be forwarded to the next processing node for further action. Please format your response in a clear and structured manner.
##### LLM2
You have received key information extracted from a math word problem. Your objective is to use this information to solve the problem step-by-step. Begin by outlining the mathematical operations needed based on the relationships identified. Perform the necessary calculations and provide the final solution, along with a brief explanation of the steps you took to arrive at this solution. Ensure that your response is clear and logical, as it will be the final output of the problem-solving process.

# Agent ID: agent-3dc2b205-3012-4422-9701-70e9c8f785b2
## Fitness: 0.6989944337295713
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node(Math Problem Analyzer)
	Llm_Node(Math Problem Analyzer) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative network of Large Language Models designed to solve complex math word problems. Each node in this network has a specific role in the problem-solving process, ensuring effective communication and integration of reasoning capabilities. Your task is to analyze the given math word problem, extract relevant information, and convey your findings to the subsequent node. You should use chain-of-thought reasoning to articulate your analysis clearly, and you are encouraged to reflect on your reasoning process to ensure accuracy. Be prepared to provide evaluative feedback if needed, to guide the next stages of problem-solving. Your ultimate goal is to derive a solution based on the provided word problem.
##### Math Problem Analyzer
You are tasked with analyzing the following math word problem. Begin by carefully reading the problem statement to extract key information, such as numbers, operations, and the relationships between them. Formulate a clear mathematical representation based on this information, including equations or expressions that accurately capture the essence of the problem. Ensure your reasoning is transparent by explaining each step of your thought process. Once you have derived the mathematical representation, summarize your findings and provide a concise explanation of how you reached this conclusion. Finally, pass this information along to the next node in the system for further processing.



Saving current state to ../data/2025-03-05_22-22-53/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:12.064653 for a total cost of 0.001742235 $
Generation 0 completed in 0:00:12.078086 with a total cost of 0.001742235 $
Budget left after first generation: 0.00225226 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 4433.73it/s]
Agent agent-02268d03-c42e-48ad-9dcb-de8afff5ba9d fitness: 0.7645708662128131, cost: 0.0
Agent agent-454327c9-af3b-4647-b488-b8e8f9d655c7 fitness: 0.573025940277384, cost: 0.0
Evaluation took 0:00:00.003494 for a total cost of 0.0 $


# Agent ID: agent-02268d03-c42e-48ad-9dcb-de8afff5ba9d
## Fitness: 0.7645708662128131
```mermaid 
graph LR
	Start_Node[start_node] --> Llm1(LLM1)
	Llm1(LLM1) --> Llm2(LLM2)
	Llm2(LLM2) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a network of specialized LLM nodes. Your task is to contribute meaningfully to the problem-solving process while communicating effectively with other LLM nodes. The process begins with the interpretation of the problem, followed by the solution generation. Each node has a distinct role: LLM1: Interpret and extract key information from the math word problem. LLM2: Perform calculations based on the extracted information and generate a solution.
##### LLM1
You are tasked with interpreting a math word problem. Read the problem carefully and identify key elements such as numbers, operations, and relationships. Provide a concise summary that outlines these key details, ensuring to clarify any ambiguities or complexities in the problem. This summary will be forwarded to the next processing node for further action. Please format your response in a clear and structured manner.
##### LLM2
You have received key information extracted from a math word problem. Your objective is to use this information to solve the problem step-by-step. Begin by outlining the mathematical operations needed based on the relationships identified. Perform the necessary calculations and provide the final solution, along with a brief explanation of the steps you took to arrive at this solution. Ensure that your response is clear and logical, as it will be the final output of the problem-solving process.

# Agent ID: agent-454327c9-af3b-4647-b488-b8e8f9d655c7
## Fitness: 0.573025940277384
```mermaid 
graph LR
	Start_Node[start_node] --> Llm1(LLM1)
	Llm1(LLM1) --> Llm2(LLM2)
	Llm2(LLM2) --> Llm3(LLM3)
	Llm3(LLM3) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a network of specialized LLM nodes. Your task is to contribute meaningfully to the problem-solving process while communicating effectively with other LLM nodes. The process begins with the interpretation of the problem, followed by outlining necessary operations and culminating in solution generation. Each node has a distinct role: LLM1: Interpret and extract key information. LLM2: Identify and outline mathematical operations. LLM3: Perform calculations and generate final solutions.
##### LLM1
You are tasked with interpreting a math word problem. Read the problem carefully and identify key elements such as numbers, operations, and relationships. Provide a concise summary that outlines these key details, ensuring to clarify any ambiguities or complexities in the problem. This summary will be forwarded to the next processing node for further action. Please format your response in a clear and structured manner.
##### LLM2
You have received key information extracted from a math word problem. Your objective is to identify the necessary mathematical operations based on the relationships identified. Provide a detailed outline of the operations and logic needed to solve the problem.
##### LLM3
Now that you have an outline of the necessary operations, perform the calculations step-by-step using the information provided. Provide the final solution along with a brief explanation of the steps you took to arrive at this solution. Ensure clarity in your response.



Saving current state to ../data/2025-03-05_22-22-53/generation=1
Starting crossover and mutation...
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.010646295000000002. Finishing process.
Error when calling azure-gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.010646295000000002
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.011756250000000003. Finishing process.
Error when calling azure-gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.011756250000000003
BudgetExceededError: Master budget limit exceeded. Limit: 0.01, New total: 0.012856800000000003. Finishing process.
Error when calling azure-gpt-4o-mini: Master budget limit exceeded. Limit: 0.01, New total: 0.012856800000000003
Error while running agent agent-dc6482db-a51a-4c64-b617-95cbbc17eebf: Recursion limit of 7 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For trou

We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [ ]:
forge.display_results(best_agents, best_fitness)

# Agent ID: agent-454327c9-af3b-4647-b488-b8e8f9d655c7
## Fitness: 0.8833838264415125
```mermaid 
graph LR
	Start_Node[start_node] --> Llm1(LLM1)
	Llm1(LLM1) --> Llm2(LLM2)
	Llm2(LLM2) --> Llm3(LLM3)
	Llm3(LLM3) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a network of specialized LLM nodes. Your task is to contribute meaningfully to the problem-solving process while communicating effectively with other LLM nodes. The process begins with the interpretation of the problem, followed by outlining necessary operations and culminating in solution generation. Each node has a distinct role: LLM1: Interpret and extract key information. LLM2: Identify and outline mathematical operations. LLM3: Perform calculations and generate final solutions.
##### LLM1
You are tasked with interpreting a math word problem. Read the problem carefully and identify key elements such as numbers, operations, and relationships. Provide a concise summary that outlines these key details, ensuring to clarify any ambiguities or complexities in the problem. This summary will be forwarded to the next processing node for further action. Please format your response in a clear and structured manner.
##### LLM2
You have received key information extracted from a math word problem. Your objective is to identify the necessary mathematical operations based on the relationships identified. Provide a detailed outline of the operations and logic needed to solve the problem.
##### LLM3
Now that you have an outline of the necessary operations, perform the calculations step-by-step using the information provided. Provide the final solution along with a brief explanation of the steps you took to arrive at this solution. Ensure clarity in your response.

